# Creating An Endpoint Hosted On An Azure Container Instance (ACI)

In [ ]:
import os
import requests
import json
import random
import numpy as np
import pandas as pd
from azureml.core import Workspace, Datastore, Dataset, Experiment, Environment, Model
from azureml.core.compute import ComputeTarget, AksCompute
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice, AksWebservice
from azureml.train.automl.run import AutoMLRun

## Initializing The Workspace & Compute Target

In [ ]:
ws = Workspace.from_config(path = 'config.json')
datastore = Datastore.get_default(ws)
compute_target = ComputeTarget(ws, 'automl-cluster')

## Initializing Azure Default Environment For Real-Time Deployment

In [ ]:
automl_environment = Environment.get(ws, 'AutoML Environment For Scoring')

In [ ]:
diabetes_regression_model = Model(workspace = ws, name = 'Diabetes-AllColumns-Regression-AutoML')
diabetes_regression_experiment = Experiment(workspace = ws, name = 'Diabetes-Sample-Regression')
diabetes_regression_run = AutoMLRun(experiment = diabetes_regression_experiment, 
                                    run_id = 'AutoML_05a05f45-6d7e-45e1-9b2a-41ff3dd9d4af')

In [5]:
best_regression_run, best_fitted_regression_model = diabetes_regression_run.get_output()

Package:azureml-automl-runtime, training version:1.52.0.post1, current version:1.51.0.post1
Package:azureml-core, training version:1.52.0, current version:1.51.0
Package:azureml-dataprep, training version:4.11.4, current version:4.10.8
Package:azureml-dataprep-rslex, training version:2.18.4, current version:2.17.12
Package:azureml-dataset-runtime, training version:1.52.0, current version:1.51.0
Package:azureml-defaults, training version:1.52.0, current version:1.51.0
Package:azureml-interpret, training version:1.52.0, current version:1.51.0
Package:azureml-mlflow, training version:1.52.0, current version:1.51.0
Package:azureml-pipeline-core, training version:1.52.0, current version:1.51.0
Package:azureml-responsibleai, training version:1.52.0, current version:1.51.0
Package:azureml-telemetry, training version:1.52.0, current version:1.51.0
Package:azureml-train-automl-client, training version:1.52.0, current version:1.51.0.post1
Package:azureml-train-automl-runtime, training version:1.

In [6]:
best_regression_run.download_file('outputs/scoring_file_v_1_0_0.py', 'Scoring Scripts/Diabetes_Real_Time_Scoring.py')

## Configuring The Endpoint To Use The Imported Python Script & Default Environment

In [7]:
inference_configuration = InferenceConfig(entry_script = 'Diabetes_Real_Time_Scoring.py', 
                                          source_directory = 'Scoring Scripts', environment = automl_environment)

## Configuring The ACI Deployment

In [8]:
aci_configuration = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1, tags = {'Project': 'Diabetes'},
                                                       description = 'Real-Time Azure Container Instance (ACI) Deployment For The Diabetes-AllColumns-Regression-AutoML Model')

In [9]:
aci_scoring_service = Model.deploy(ws, 'diabetes-scoring-aci', [diabetes_regression_model], inference_configuration,
                                  aci_configuration, overwrite = True)
aci_scoring_service.wait_for_deployment(show_output = True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2023-11-22 10:31:12+00:00 Creating Container Registry if not exists.
2023-11-22 10:31:12+00:00 Registering the environment.
2023-11-22 10:31:13+00:00 Use the existing image.
2023-11-22 10:31:13+00:00 Generating deployment configuration.
2023-11-22 10:31:13+00:00 Submitting deployment to compute.
2023-11-22 10:31:15+00:00 Checking the status of deployment diabetes-scoring-aci..
2023-11-22 10:36:19+00:00 Checking the status of inference endpoint diabetes-scoring-aci.
Succeeded
ACI service creation operation finished, operation "Succeeded"


## Testing The Real-Time Scoring Endpoint

### Creating Dummy Diabetes Data

In [10]:
AGE_range = np.arange(19, 79, 1)
BMI_range = np.arange(18.0, 42.2, 0.1)
BP_range = np.arange(62, 133, 1)
S1_range = np.arange(97, 301, 1)
S2_range = np.arange(41.6, 242.4, 0.1)
S3_range = np.arange(22, 99, 1)
S4_range = np.arange(2, 9.09, 0.01)
S5_range = np.arange(3.26, 6.11, 0.001)
S6_range = np.arange(58, 124, 1)

columns = ['AGE', 'SEX', 'BMI', 'BP', 'S1', 'S2', 'S3', 'S4', 'S5', 'S6']
diabetes_df = pd.DataFrame(columns = columns)
diabetes_list = []
for i in range(50):
    values = [random.choice(AGE_range), random.choice([1, 2]), random.choice(BMI_range), random.choice(BP_range),
              random.choice(S1_range), random.choice(S2_range), random.choice(S3_range), random.choice(S4_range),
              random.choice(S5_range), random.choice(S6_range)]
    diabetes_list.append(pd.DataFrame(dict(zip(columns, values)), index = [0]))
    
diabetes_df = diabetes_df.append(diabetes_list, True)
diabetes_df

,AGE,SEX,BMI,BP,S1,S2,S3,S4,S5,S6
0,55,2,31.70,105,286,133.80,75,6.68,5.08,92
1,62,2,27.60,105,164,122.00,33,2.13,3.44,67
2,60,2,39.70,108,236,88.20,22,5.05,5.63,59
3,46,2,37.80,70,289,160.10,76,8.57,6.04,99
4,27,1,36.90,107,240,53.80,32,6.98,5.17,64
5,25,2,23.10,96,225,85.50,98,2.20,4.65,59
6,64,2,34.70,108,172,124.00,73,5.95,4.59,79
7,21,2,35.20,69,238,91.10,67,3.65,4.94,123
8,58,2,26.00,95,276,213.70,52,4.59,5.80,72
9,22,2,29.30,84,254,93.30,93,5.10,4.07,116


In [11]:
Dataset.Tabular.register_pandas_dataframe(diabetes_df, datastore, 'Diabetes Sample Dataset For Scoring')

Validating arguments.
Arguments validated.
Successfully obtained datastore reference and path.
Uploading file to managed-dataset/39639b76-d86c-440e-a115-fe2292ec5452/
Successfully uploaded file to datastore.
Creating and registering a new dataset.
Successfully created and registered a new dataset.


{
  "source": [
    "('workspaceblobstore', 'managed-dataset/39639b76-d86c-440e-a115-fe2292ec5452/')"
  ],
  "definition": [
    "GetDatastoreFiles",
    "ReadParquetFile",
    "DropColumns"
  ],
  "registration": {
    "id": "aa113bc6-2e11-4c36-a07b-0303ae95235b",
    "name": "Diabetes Sample Dataset For Scoring",
    "version": 6,
    "workspace": "Workspace.create(name='auotml-example-workspace', subscription_id='0c19fc19-85fd-4aa4-b133-61dd20fa93df', resource_group='edwin.spartan117-rg')"
  }
}

In [12]:
test_input_json = '{"data":' + diabetes_df.to_json(orient = 'records') + '}'
test_input_json

'{"data":[{"AGE":55,"SEX":2,"BMI":31.7,"BP":105,"S1":286,"S2":133.8,"S3":75,"S4":6.68,"S5":5.084,"S6":92},{"AGE":62,"SEX":2,"BMI":27.6,"BP":105,"S1":164,"S2":122.0,"S3":33,"S4":2.13,"S5":3.44,"S6":67},{"AGE":60,"SEX":2,"BMI":39.7,"BP":108,"S1":236,"S2":88.2,"S3":22,"S4":5.05,"S5":5.627,"S6":59},{"AGE":46,"SEX":2,"BMI":37.8,"BP":70,"S1":289,"S2":160.1,"S3":76,"S4":8.57,"S5":6.043,"S6":99},{"AGE":27,"SEX":1,"BMI":36.9,"BP":107,"S1":240,"S2":53.8,"S3":32,"S4":6.98,"S5":5.172,"S6":64},{"AGE":25,"SEX":2,"BMI":23.1,"BP":96,"S1":225,"S2":85.5,"S3":98,"S4":2.2,"S5":4.649,"S6":59},{"AGE":64,"SEX":2,"BMI":34.7,"BP":108,"S1":172,"S2":124.0,"S3":73,"S4":5.95,"S5":4.593,"S6":79},{"AGE":21,"SEX":2,"BMI":35.2,"BP":69,"S1":238,"S2":91.1,"S3":67,"S4":3.65,"S5":4.943,"S6":123},{"AGE":58,"SEX":2,"BMI":26.0,"BP":95,"S1":276,"S2":213.7,"S3":52,"S4":4.59,"S5":5.796,"S6":72},{"AGE":22,"SEX":2,"BMI":29.3,"BP":84,"S1":254,"S2":93.3,"S3":93,"S4":5.1,"S5":4.074,"S6":116},{"AGE":43,"SEX":2,"BMI":32.4,"BP":75,"S1"

In [13]:
aci_deployment_response = requests.post('YOUR_ACI_SCORING_ENDPOINT_URL',
                                        test_input_json, headers = {'Content-Type': 'application/json'})
print(aci_deployment_response.text)

"{\"result\": [173.09544864825628, 108.05837095957796, 227.1414763027679, 223.03179214957422, 206.62495198708308, 85.90215893061453, 224.05624613273244, 175.90584721960778, 185.89357742838726, 89.736462384324, 144.33157166290863, 260.87767019558464, 107.11394995284587, 281.4128296767971, 236.72259103002943, 224.10013260846836, 187.91435391857985, 141.62083333177026, 230.98734329302792, 202.5593239587996, 244.75008267478384, 256.1935402847104, 186.57202799617278, 182.39330609794536, 68.91554231419309, 125.32396310821201, 113.01325693274367, 238.71042338040203, 195.09775221503594, 78.6011685453675, 113.60254155945101, 227.59922842168345, 256.64354755284864, 175.93006441895253, 71.25659666648441, 157.1821911711245, 194.62315267215226, 43.72175664241207, 256.4186372697365, 269.21769406374995, 144.1259957675329, 247.2190376598925, 183.1738513424749, 263.07072524837923, 183.10145886904877, 109.33962483544364, 221.48430019043417, 326.486125203474, 225.76274593971118, 120.80945204588117]}"


# Creating An Azure Kubernetes Service (AKS) Cluster

In [23]:
provisioning_configuration = AksCompute.provisioning_configuration(
    vm_size = 'Standard_DS3_v2', agent_count = 2, cluster_purpose = 'DevTest', location = 'eastasia')

aks_target = ComputeTarget.create(workspace = ws, name = 'AKS-Cluster-Code', 
                                  provisioning_configuration = provisioning_configuration)
aks_target.wait_for_completion(show_output = True)

InProgress...................................................................................
SucceededProvisioning operation finished, operation "Succeeded"


# Creating An Endpoint Hosted On An AKS Cluster

## Configuring The Deployment Of The AKS Cluster

In [24]:
aks_configuration = AksWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1, tags = {'Project': 'Diabetes'},
                                                       description = 'Real-Time Deployment Of The Diabetes-AllColumns-Regression-AutoML Model On An Azure Kubernetes Service (AKS) Cluster')

In [25]:
aks_cluster = AksCompute(ws, 'AKS-Cluster-Code')

aks_service = Model.deploy(ws, 'diabetes-scoring-aks', [diabetes_regression_model], inference_configuration, 
                           aks_configuration, aks_cluster, overwrite = True)
aks_service.wait_for_deployment(show_output = True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2023-11-22 11:15:34+00:00 Creating Container Registry if not exists.
2023-11-22 11:15:34+00:00 Registering the environment.
2023-11-22 11:15:35+00:00 Use the existing image.
2023-11-22 11:15:37+00:00 Creating resources in AKS..
2023-11-22 11:15:39+00:00 Submitting deployment to compute.
2023-11-22 11:15:39+00:00 Checking the status of deployment diabetes-scoring-aks..
2023-11-22 11:17:04+00:00 Checking the status of inference endpoint diabetes-scoring-aks.
Succeeded
AKS service creation operation finished, operation "Succeeded"


In [27]:
aks_deployment_response = requests.post(<YOUR_AKS_SCORING_ENDPOINT_URL>, test_input_json, 
                                        headers = {'Content-Type': 'application/json', 'Authorization': 'Bearer <YOUR_API_KEY>'})
print(aks_deployment_response.text)

"{\"result\": [173.09544864825628, 108.05837095957796, 227.1414763027679, 223.03179214957422, 206.62495198708308, 85.90215893061453, 224.05624613273244, 175.90584721960778, 185.89357742838726, 89.736462384324, 144.33157166290863, 260.87767019558464, 107.11394995284587, 281.4128296767971, 236.72259103002943, 224.10013260846836, 187.91435391857985, 141.62083333177026, 230.98734329302792, 202.5593239587996, 244.75008267478384, 256.1935402847104, 186.57202799617278, 182.39330609794536, 68.91554231419309, 125.32396310821201, 113.01325693274367, 238.71042338040203, 195.09775221503594, 78.6011685453675, 113.60254155945101, 227.59922842168345, 256.64354755284864, 175.93006441895253, 71.25659666648441, 157.1821911711245, 194.62315267215226, 43.72175664241207, 256.4186372697365, 269.21769406374995, 144.1259957675329, 247.2190376598925, 183.1738513424749, 263.07072524837923, 183.10145886904877, 109.33962483544364, 221.48430019043417, 326.486125203474, 225.76274593971118, 120.80945204588117]}"
